<h3>Genetic Algorithm</h3>

Genetic algorithms simulate the process of natural selection [ is a mechanism of evolution that explains how populations of living organisms adapt and change over time. ]. In simpler words, they simulate 'survival of the fittest' among individuals of consecutive generations to solve a problem.

<h5>Libraries and variables</h5>

In [1]:
import random
import math

from tqdm import trange,tqdm #visualizing the loops

GLOBAL_CHROMOSOME_LENGTH=6
GLOBAL_RANDINT_MIN=0
GLOBAL_RANDINT_MAX=5
GLOBAL_POP_LIMIT=40             #it is a X-X grid

GLOBAL_MUTATOR_CHANCE=10
GLOBAL_MUTATOR_RANGE=1000

GLOBAL_EPOCH_COUNT=4
GLOBAL_FITTING_DATA=[i for i in range(GLOBAL_CHROMOSOME_LENGTH)]

<h5>Gene Class Creation</h5>

In [2]:
class Gene:
    chromosomes=[]
    saved_chromosome_length=0
    saved_randint_min=0
    saved_randint_max=0
    fitness_factor=9999
    def __init__(self,fitting_data=GLOBAL_FITTING_DATA,
                 chromosome_length=GLOBAL_CHROMOSOME_LENGTH,randint_min=GLOBAL_RANDINT_MIN,randint_max=GLOBAL_RANDINT_MAX,
                 rand_gene=True,initial_eval=False): #constructor
        self.chromosomes=[]
        self.saved_chromosome_length=chromosome_length
        self.saved_randint_min=randint_min
        self.saved_randint_max=randint_max
        if rand_gene:
            self.create_random_gene(chromosome_length,randint_min,randint_max)
            if initial_eval:
                self.fitness_evaluation(fitting_data)
        pass    
    def create_random_gene(self,chromosome_length,randint_min,randint_max): #initializes the gene based on chromosome length and randint range
        for i in range(chromosome_length):
            self.chromosomes.append(random.randint(randint_min,randint_max))
        pass
    def crossover_with_gene(self,input_gene,mutate=True): #crosses over with another gene, uniform crossover, includes mutation
        child=Gene(chromosome_length=self.saved_chromosome_length,rand_gene=False)
        for i in range(self.saved_chromosome_length):
            if bool(random.getrandbits(1)):
                child.chromosomes.append(self.chromosomes[i])
            else:
                child.chromosomes.append(input_gene.chromosomes[i])
        child.random_mutator(mutate)
        return child
    def equate_with_gene(self,input_gene,mutate=False): #equates over an equation with another gene, includes mutation
        child=Gene(chromosome_length=self.saved_chromosome_length,rand_gene=False)
        for i in range(self.saved_chromosome_length):
            child.chromosomes.append(int(self.chromosomes[i])+int(input_gene.chromosomes[i])) #equation parser goes here
        child.random_mutator(mutate)
        return child
    def random_mutator(self,mutate): #imitates mutation based on mutation chances 
        if mutate:
            if random.randint(1,GLOBAL_MUTATOR_RANGE)<=GLOBAL_MUTATOR_CHANCE:
                self.chromosomes[random.randint(0,self.saved_chromosome_length-1)]=random.randint(self.saved_randint_min,self.saved_randint_max)
    def fitness_evaluation(self,fitting_data): #generates the fitness factor based on fitting_data
        self.fitness_factor=self.saved_chromosome_length
        for i in range(self.saved_chromosome_length):
            if self.chromosomes[i]==fitting_data[i]:
                self.fitness_factor-=1
        return self.fitness_factor
    def display(self): #returns a string with the chromosomes and fitness factor as output
        return str('Chromosomes: '+str(self.chromosomes)+' Fitness Factor: '+str(self.fitness_factor))


<h5>Gene List Class</h5>

In [3]:
class Gene_List: #WIP
    #initialization of lists
    fitting_data=[]
    generation_list=[]
    index_list=[]
    gene_list=[]
    
    def __init__(self,fitting_data=GLOBAL_FITTING_DATA,epoch=GLOBAL_EPOCH_COUNT,pop_limit=GLOBAL_POP_LIMIT,
                 chromosome_length=GLOBAL_CHROMOSOME_LENGTH,randint_min=GLOBAL_RANDINT_MIN,randint_max=GLOBAL_RANDINT_MAX,
                 crossover_method='cartesian_prod',crossover_prune_method='random_survivor',
                 stop=True,verbose=True,mutate=True,rand_gene=True,initial_eval=True,manual=False):
        self.fitting_data=fitting_data
        self.generation_list=[]                         #emptying
        self.index_list=[]                              #emptying
        self.gene_list=self.Gene_List_Gen(pop_limit=pop_limit,                      #generate initial gene list [Square]
                                          chromosome_length=chromosome_length,randint_min=randint_min,randint_max=randint_max,
                                          rand_gene=rand_gene,initial_eval=initial_eval)             
        self.generation_list.append(self.gene_list)     #appends the initial gene list to the generation list, it is generation zero
        if manual==False:
            self.List_Iterator(stop,epoch,verbose,crossover_method,crossover_prune_method,pop_limit,mutate)                            #calls the auto iterator
        pass
    def List_Iterator(self,stop,epoch,verbose,crossover_method,crossover_prune_method,pop_limit,mutate): #Auto Iterator
        for i in trange(epoch,desc='Epoch'):
            if self.index_list==[] or stop==False:                                                             #if no fit gene is found
                self.generation_list.append(self.Gene_List_Cross(input_list=self.generation_list[i],pop_limit=pop_limit,
                                                                 crossover_method=crossover_method,crossover_prune_method=crossover_prune_method,
                                                                 verbose=verbose,mutate=mutate))    #generate new generation
                self.Gene_List_Eval_I(self.generation_list[i],verbose,stop)                        #evaluate the new generation
                if self.index_list!=[]:                                                         #if fit gene found
                    self.GLV(self.generation_list[i],self.index_list)                           #display fit gene
                    return                                                                     #end loop
    def Gene_List_Gen(self,pop_limit=GLOBAL_POP_LIMIT, #Gene List Generator [Initial] [Standalone+Iterator]
                      chromosome_length=GLOBAL_CHROMOSOME_LENGTH,randint_min=GLOBAL_RANDINT_MIN,randint_max=GLOBAL_RANDINT_MAX,
                      rand_gene=True,initial_eval=False): 
        result=[]
        temp=[]
        for i in trange(pop_limit,desc='Generating genes'):
            temp=[]
            for j in range(pop_limit):
                temp.append(Gene(fitting_data=self.fitting_data,
                                 chromosome_length=chromosome_length,randint_min=randint_min,randint_max=randint_max,
                                 rand_gene=rand_gene,initial_eval=initial_eval))
            result.append(temp)
        return result
    def Gene_List_Cross(self,input_list,pop_limit=GLOBAL_POP_LIMIT, #Gene List Crossover Function [Standalone+Iterator]
                        crossover_method='cartesian_prod',crossover_prune_method='random_survivor',
                        verbose=True,mutate=True): 
        result=[]
        cross=[]
        temp=[]
        if verbose:
            cross_verbose='Crossover function executing'
            rand_verbose='Selecting random survivors'
            if crossover_method=='cartesian_prod':
                for i in trange(len(input_list),desc=cross_verbose):
                    for j in range(len(input_list)):
                        temp=[]
                        for k in range(len(input_list)):
                            for l in range(len(input_list)):
                                temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate))
                        cross.append(temp)
            if crossover_method=='equate-cartesian_prod':
                for i in trange(len(input_list),desc=cross_verbose):
                    for j in range(len(input_list)):
                        temp=[]
                        for k in range(len(input_list)):
                            for l in range(len(input_list)):
                                temp.append(input_list[i][j].equate_with_gene(input_list[k][l],mutate))
                        cross.append(temp)
            if crossover_prune_method=='random_survivor':
                surv_list=[]
                for i in trange(pop_limit,desc=rand_verbose):
                    temp=[]
                    for j in range(pop_limit):
                        surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                        surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                        temp.append(cross[surv_x][surv_y])
                    surv_list.append(temp)
                result=surv_list
            return result
        else:
            if crossover_method=='cartesian_prod':
                for i in range(len(input_list)):
                    for j in range(len(input_list)):
                        temp=[]
                        for k in range(len(input_list)):
                            for l in range(len(input_list)):
                                temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate))
                        cross.append(temp)
            if crossover_method=='equate-cartesian_prod':
                for i in range(len(input_list)):
                    for j in range(len(input_list)):
                        temp=[]
                        for k in range(len(input_list)):
                            for l in range(len(input_list)):
                                temp.append(input_list[i][j].equate_with_gene(input_list[k][l],mutate))
                        cross.append(temp)
            if crossover_prune_method=='random_survivor':
                surv_list=[]
                for i in range(pop_limit):
                    temp=[]
                    for j in range(pop_limit):
                        surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                        surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                        temp.append(cross[surv_x][surv_y])
                    surv_list.append(temp)
                result=surv_list
            return result
    def Gene_List_Eval(self,input_list, #Gene List Fitness Evaluation Function [Standalone+Iterator]
                       verbose=True,stop=False): 
        result=[]
        temp=[]
        if verbose:
            eval_str='Evaluating genes'
            for i in trange(len(input_list),desc=eval_str):
                for j in range(len(input_list[0])):
                    if input_list[i][j].fitness_evaluation(self.fitting_data)==0:
                        temp=[]
                        temp.append(i)
                        temp.append(j)
                        result.append(temp)
                        if stop:
                            self.index_list=result
                            return
            self.index_list=result
            pass
        else:
            for i in range(len(input_list)):
                for j in range(len(input_list[0])):
                    if input_list[i][j].fitness_evaluation(self.fitting_data)==0:
                        temp=[]
                        temp.append(i)
                        temp.append(j)
                        result.append(temp)
                        if stop:
                            self.index_list=result
                            return
            self.index_list=result
            pass
    def Gene_List_Viewer(self): #Gene List Viewer [Standalone]
        for i in tqdm(self.index_list,desc='Displaying genes'):
            print('Gene '+str(i)+' '+self.gene_list[i[0]][i[1]].display())
    def GLV(self,input_list,index_list): #Gene List Viewer [Iterator Version]
        for i in tqdm(self.index_list,desc='Displaying genes'):
            print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())
    def GLCEF(self,input_list,stop=False): #Gene List Crossover & Evaluation Function [Iterator Version] [Generation Generator] [Deprecated]
        result=[]
        temp=[]
        for i in trange(len(input_list),desc='Crossover & Evaluation function executing'):
            for j in range(len(input_list)):
                temp=[]
                for k in range(len(input_list)):
                    for l in range(len(input_list)):
                        child=input_list[i][j].crossover_with_gene(input_list[k][l])
                        temp.append(child)
                        child.fitness_evaluation(self.fitting_data)
                        if child.fitness_factor==0:
                            temp2=[]
                            temp2.append(i)
                            temp2.append(j)
                            self.index_list.append(temp2)
                            if stop:
                                pass
                                
                result.append(temp)
        return result
    def Gene_List_Cross_I(self,input_list,verbose,crossover_prune_method,pop_limit,mutate): #Gene List Crossover Function [Iterator Version] [Deprecating]
        result=[]
        cross=[]
        temp=[]
        if verbose:
            cross_verbose='Crossover function executing'
            rand_verbose='Selecting random survivors'
            for i in trange(len(input_list),desc=cross_verbose):
                for j in range(len(input_list)):
                    temp=[]
                    for k in range(len(input_list)):
                        for l in range(len(input_list)):
                            temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate))
                    cross.append(temp)
            if crossover_prune_method=='random_survivor':
                surv_list=[]
                for i in trange(pop_limit,desc=rand_verbose):
                    temp=[]
                    for j in range(pop_limit):
                        surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                        surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                        temp.append(cross[surv_x][surv_y])
                    surv_list.append(temp)
                result=surv_list
            return result
        else: #verbose=False
            for i in range(len(input_list)):
                for j in range(len(input_list)):
                    temp=[]
                    for k in range(len(input_list)):
                        for l in range(len(input_list)):
                            temp.append(input_list[i][j].crossover_with_gene(input_list[k][l],mutate))
                    cross.append(temp)
            if crossover_prune_method=='random_survivor':
                surv_list=[]
                for i in range(pop_limit):
                    temp=[]
                    for j in range(pop_limit):
                        surv_x=random.randint(0,math.pow(pop_limit,2)-1)
                        surv_y=random.randint(0,math.pow(pop_limit,2)-1)
                        temp.append(cross[surv_x][surv_y])
                    surv_list.append(temp)
                result=surv_list
            return result
    def Gene_List_Eval_I(self,input_list,verbose,stop): #Gene List Fitness Evaluation Function [Iterator Version] [Deprecating]
        result=[]
        temp=[]
        if verbose:
            eval_str='Evaluating genes'
            for i in trange(len(input_list),desc=eval_str):
                for j in range(len(input_list[0])):
                    if input_list[i][j].fitness_evaluation(self.fitting_data)==0:
                        temp=[]
                        temp.append(i)
                        temp.append(j)
                        result.append(temp)
                        if stop:
                            self.index_list=result
                            return
            self.index_list=result
            pass
        else:
            for i in range(len(input_list)):
                for j in range(len(input_list[0])):
                    if input_list[i][j].fitness_evaluation(self.fitting_data)==0:
                        temp=[]
                        temp.append(i)
                        temp.append(j)
                        result.append(temp)
                        if stop:
                            self.index_list=result
                            return
            self.index_list=result
            pass

In [4]:
GList=Gene_List(manual=True,chromosome_length=1,randint_min=1,randint_max=10,rand_gene=True,pop_limit=4,mutate=False,initial_eval=False)
GList2=Gene_List(manual=True,chromosome_length=1,randint_min=-10,randint_max=10,rand_gene=True,pop_limit=4,mutate=False,initial_eval=False)

Generating genes: 100%|██████████| 4/4 [00:00<00:00, 3999.34it/s]


In [5]:
GList.Gene_List_Cross(GList2.generation_list[0],mutate=False,pop_limit=4)

Selecting random survivors: 100%|██████████| 4/4 [00:00<?, ?it/s]


[[<__main__.Gene at 0x1baff343c50>,

In [6]:
GList3=Gene_List(manual=True,rand_gene=False)

Generating genes: 100%|██████████| 40/40 [00:00<00:00, 39993.36it/s]


In [7]:
GList3.generation_list.append(GList.Gene_List_Cross(GList2.generation_list[0],mutate=False,pop_limit=4,crossover_method='equate-cartesian_prod',verbose=False))

In [9]:
GList3.generation_list[1][0][0].chromosomes[0]

1

In [10]:
GList4=Gene_List(epoch=100,verbose=False)

Epoch:   5%|▌         | 5/100 [00:39<12:30,  7.90s/it]

Gene [18, 7] Chromosomes: [0, 1, 2, 3, 4, 5] Fitness Factor: 0


<h6><i>Everything below is deprecated</i></h6>

In [ ]:
def GLG(): #Gene List Generator [Initial]
    result=[]
    temp=[]
    for i in trange(GLOBAL_POP_LIMIT,desc='Generating genes'):
        temp=[]
        for j in range(GLOBAL_POP_LIMIT):
            temp.append(Gene(rand_gene=True))
        result.append(temp)
    return result

In [ ]:
def GLCF(input_list): #Gene List Crossover Function
    result=[]
    temp=[]
    for i in trange(len(input_list),desc='Crossover function executing'):
        for j in range(len(input_list)):
            temp=[]
            for k in range(len(input_list)):
                for l in range(len(input_list)):
                    temp.append(input_list[i][j].crossover_with_gene(input_list[k][l]))
            result.append(temp)
    return result

In [ ]:
def GLFEF(input_list,fitting_data,stop=False): #Gene List Fitness Evaluation Function
    result=[]
    temp=[]
    for i in trange(len(input_list),desc='Evaluating genes'):
        for j in range(len(input_list[0])):
            if input_list[i][j].fitness_evaluation(fitting_data)==0:
                temp=[]
                temp.append(i)
                temp.append(j)
                result.append(temp)
                if stop:
                    return result
    return result

In [ ]:
def GLV(input_list,index_list): #Gene List Viewer
    for i in tqdm(index_list,desc='Displaying genes'):
        print('Gene '+str(i)+' '+input_list[i[0]][i[1]].display())

In [ ]:
def GCLRS(input_list): #Gene Crossover List Random Sampler
    len_list=len(input_list)
    parent_1=random.randint(0,len_list-1)
    parent_2=random.randint(0,len_list-1)
    print('\nParent 1 ['+str(parent_1)+']:'+input_list[parent_1][parent_1].display())
    print('\nParent 2 ['+str(parent_2)+']:'+input_list[parent_1][parent_1].display())
    print('\nChild 1 (P1xP2): '+input_list[parent_1][parent_2].display())
    print('\nChild 2 (P2xP1): '+input_list[parent_2][parent_1].display())

In [ ]:
fitting_data=[x for x in range(GLOBAL_CHROMOSOME_LENGTH)]
list_of_genes=GLG()
first_gen=GLCF(list_of_genes)
index_list=GLFEF(first_gen,fitting_data=fitting_data,stop=True)
GLV(first_gen,index_list)

Displaying genes: 100%|██████████| 1/1 [00:00<00:00, 1000.31it/s]

Gene [72, 835] Chromosomes: [0, 1, 2, 3, 4, 5] Fitness Factor: 0


In [ ]:
run=True
gene_list=GLG()
generation_list=[]
generation_list.append(gene_list)
index_list=[]
while run==False:
    for i in trange(GLOBAL_EPOCH_COUNT):
        if index_list==[]:
            generation_list.append(GLCF(generation_list[i]))
            index_list=GLFEF(generation_list[i],fitting_data,stop=True)
        if index_list!=[]:
            GLV(generation_list[i],index_list)
            run=False
            break
    run=False

Generating genes: 100%|██████████| 40/40 [00:00<00:00, 7997.15it/s]


In [ ]:
GCLRS(first_gen)


Parent 1 [1]:Chromosomes: [5, 3, 2, 1, 3, 1] Fitness Factor: 5

Parent 2 [1270]:Chromosomes: [5, 3, 2, 1, 3, 1] Fitness Factor: 5

Child 1 (P1xP2): Chromosomes: [4, 4, 2, 1, 2, 1] Fitness Factor: 5

Child 2 (P2xP1): Chromosomes: [5, 3, 1, 1, 3, 1] Fitness Factor: 9999


In [ ]:
class Gene_ID_Gen: #deprecated
    gene_id_tracker=0
    def generate(self):
        gene_id=self.gene_id_tracker+1
        self.gene_id_tracker=self.gene_id_tracker+1
        return gene_id
    def latest(self):
        return self.gene_id_tracker

ID_Gen=Gene_ID_Gen()